#**APRENDIZAJE AUTOMÁTICO - MIGUEL MARINES**
##**<u>Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación</u>**
---
---


In [ ]:
# Google Drive en Google Colab.
# Acceso a los archivos y directorios almacenados en Google Drive desde un notebook de Colab.
from google.colab import drive
drive.mount('/content/drive')

# Importación de la librería "os" para manipular rutas de archivos.
# Ruta del directorio que se desea establecer como directorio de trabajo.
# Se cambia el directorio de trabajo actual al especificado en la variable "DIR".
import os
DIR = "/content/drive/MyDrive/Machine_Learning"
os.chdir(DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Librerías
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD

### **Liga de Datos de la UCI "Restaurant & Consumer Data".**
### **Se descarga "RCdata.zip" y de ahí se utilizarán los archivos "rating_final.csv" y "geoplaces2.csv".**

https://archive.ics.uci.edu/dataset/232/restaurant+consumer+data



In [ ]:
# Carga de datos en DataFrames.
data1 = pd.read_csv("rating_final.csv", header='infer', sep=",")
data2 = pd.read_csv("geoplaces2.csv", header='infer',  encoding='latin-1')

# Dimenciones de los DatFrames.
print(data1.shape, data2.shape)

(1161, 5) (130, 21)


## **Ejercicio - 1**

### **Se explica cuál es el propósito de usar el argumento "latin-1" al cargar el segundo archivo.**

El propósito de usar "latin-1" es garantizar que los caracteres especiales presentes en el archivo se decodifiquen y representen correctamente en la estructura de datos de Python, en este caso, un DataFrame de Pandas.

Se hace uso de "latin-1" en el segundo texto en específico, ya que contiene una gran cantidad de texto.

La codificación Latin-1, también conocida como ISO 8859-1, es una codificación de caracteres de un solo byte que puede representar caracteres de muchos idiomas europeos occidentales que utilizan el alfabeto latino, incluyendo los caracteres acentuados y especiales utilizados en idiomas como el español, francés, alemán, italiano, entre otros.

In [ ]:
# Del primer archivo se obtiene una matriz con 3 evaluaciones de los restaurantes: general, comida y servicio.
# Las evaluaciones pueden ser 0, 1 o 2. (0 es la menor calificación y 2 es la mayor calificación)
data1.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [ ]:
# Del segundo archivo se obtiene la información diversa de cada restaurante.
data2.head(2)

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none


In [ ]:
# Del DataFrame data1 no se requiere "rating" y de data2 solo se necesita "placeID" y "name".
# Se definen las listas y matrices con los factores que se necesitan.

lista_data1 = ['userID','placeID','food_rating','service_rating']
lista_data2 = ['placeID','name']

data1a = data1[lista_data1]
data2a = data2[lista_data2]

## **Ejercicio - 2**

In [ ]:
# Se define el DataFrame que conjunta la información de los dos DataFrames data1a y data2a en uno solo
# a través de la columna común "placeID".

# DataFrames combinados utilizando la columna común "placeID".
df_combinado = pd.merge(data1a, data2a, on='placeID')


# Se despliega la dimensión y los primeros renglones del DataFrame.
print(df_combinado.shape)
df_combinado.head()

(1161, 5)


,userID,placeID,food_rating,service_rating,name
0,U1077,135085,2,2,Tortas Locas Hipocampo
1,U1108,135085,2,1,Tortas Locas Hipocampo
2,U1081,135085,2,1,Tortas Locas Hipocampo
3,U1056,135085,2,2,Tortas Locas Hipocampo
4,U1134,135085,1,2,Tortas Locas Hipocampo


## **Ejercicio - 3**

In [ ]:
# Se define la matriz de utilidad cuyos renglones serán los nombres de los restaurantes,
# las columnas los IDs de los usuarios y las entradas la evaluación de la comida (food_rating).

# Matriz de utilidad.
UtMx_food = df_combinado.pivot_table(values='food_rating', index='name', columns='userID', fill_value=0)

# Mostrar resultados.
print('Dimensión de la Matriz de Utilidad sobre la evaluación de la comida:')
print('(restaurantes, usuarios) =', (UtMx_food.shape))
UtMx_food.head()

Dimensión de la Matriz de Utilidad sobre la evaluación de la comida:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,2,0,...,0,0,0,0,0,1,0,0,0,0
Cafe Chaires,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## **Ejercicio - 4**

### **En la Factorización SVD la cantidad de valores singulares será menor o igual al menor valor de los renglones o columnas de la matriz de utilidad UtMx.**

### **La factorización SVD de una matriz $A$ tiene la forma:**

$A_{m\times n} = U_{m\times m}\Sigma_{m\times n}V_{n\times n}^T$

In [ ]:
# Se aplica la transformación SVD para obtener la matriz de las variables
# latentes de los restaurantes en relación a la evalución de los usuarios con
# respecto a la comida. Se deberá obtener la factorización SVD con el máximo de
# componentes y por lo tanto el máximo de valores singulares posible.
# Se asigna a la variable "nc_food" el valor del máximo número de valores singulares
# que se puede obtener de la utilidad que se generó en el ejercicio anterior.


# Número de componentes.
nc_food = UtMx_food.shape[1] - 1


# Se inicializa y aplica la factorización SVD en relación a la evaluación de la comida,
# para determinar la cantidad de componentes que se pueden utilizar con un 90% de explicación
# de la variabilidad de dichos componentes.
SVD_food = TruncatedSVD(n_components=nc_food)
SVD_food.fit(UtMx_food)


# Se determina la cantidad de componentes que explican un 90%
# de la variabilidad acumulada de cada componente.
for j in range(nc_food):
  if SVD_food.explained_variance_ratio_[0:j].sum() > 0.90:
    break

# Se usa esta cantidad de componentes para las recomendaciones basadas
# en la calificación de la comida:
N_food = j-1

# Mostrar resultados.
print('Total de valores singulares basados en la evaluación de la comida:', nc_food)
print('Valor de truncamiento al 90% de dicha variabilidad:',N_food)

Total de valores singulares basados en la evaluación de la comida: 137
Valor de truncamiento al 90% de dicha variabilidad: 51


In [ ]:
# Se usa la métrica de correlación de Pearson, para obtener las 10 mejores
# recomendaciones de este modelo no supervisado con base a la información de
# alguien que desea obtener las similidades con el restaurante "Restaurante Pueblo
# Bonito" y con la cota de truncamiento obtenida en el ejercicio anterior.
# Para ello se obtienen las correlaciones positivas y de ahí se deberán seleccionar
# las diez mejores.

# Factorización SVD.
SVD_food = TruncatedSVD(n_components=N_food)
resultant_matrix_food = SVD_food.fit_transform(UtMx_food)

# Matriz de correlación de Pearson
corr_mat_food = np.corrcoef(resultant_matrix_food)

# Restaurante de referencia.
restaurante_de_referencia = "Restaurante Pueblo Bonito"
nombres_rest = UtMx_food.T.columns  # Nombres de restaurantes.
idx_rest = list(nombres_rest).index(restaurante_de_referencia)
corr_rest = corr_mat_food[idx_rest] # Vector de Correlación del Restaurante Pueblo Bonito.

# Busqueda de las correlaciones positivas.
idx = (corr_rest>0)
mejores_sim_food = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_food.append((corr_rest[idx][i], nombres_rest[idx][i]))

# Mostrar resultados.
print('Total de similaridades positivas encontradas:', len(mejores_sim_food))
print('Algunos de los resultados encontrados:')
mejores_sim_food[0:7]

Total de similaridades positivas encontradas: 73
Algunos de los resultados encontrados:


[(0.006575940254159448, 'Arrachela Grill'),
 (0.023194363775655653, 'Cabana Huasteca'),
 (0.030990487977367386, 'Cafe Chaires'),
 (0.017173203528347537, 'Cafeteria cenidet'),
 (0.22534230418759982, 'Cafeteria y Restaurant El Pacifico'),
 (0.01930573766406098, 'Cenaduria El RincÃ³n de Tlaquepaque'),
 (0.008138058263762411, 'Chilis Cuernavaca')]

## **Ejercicio - 5**

In [ ]:
# Se ordena la lista de recomendaciones "mejores_sim_food" encontrada en el paso
# anterior de manera descendente y se llama "mejores_sim_food_ordenadas".

# Lista de recomendaciones ordenadas.
#mejores_sim_food_ordenadas = sorted(mejores_sim_food, key=lambda x:x[0], reverse=True)
mejores_sim_food_ordenadas = sorted(mejores_sim_food, reverse=True)


# Se despliegan las 10 mejores similitudes encontradas de manera descendente.
print('Similitudes con base a la evaluación de la comida con mayores valores de correlación:\n')
for k in range(1,11):
  print('%d> %s' % (k, mejores_sim_food_ordenadas[k]))

Similitudes con base a la evaluación de la comida con mayores valores de correlación:

1> (0.7917728233193868, 'Restaurante la Estrella de Dima')
2> (0.7006610080174676, 'pizza clasica')
3> (0.6682354042725505, 'Restaurante Guerra')
4> (0.5439090580647232, 'El Club')
5> (0.3679806560572125, 'Pizzeria Julios')
6> (0.31032069772960236, 'Restaurant Oriental Express')
7> (0.3102801045185013, 'Hamburguesas saul')
8> (0.30665568060688375, 'Restaurante El Cielo Potosino')
9> (0.28203391140962764, 'Rincon Huasteco')
10> (0.26877438729999414, 'la Cochinita Pibil Restaurante Yucateco')


## **Ejercicio - 6**

### **De manera análoga y usando ahora la evaluación con respecto al servicio (service_rating), se encuentran ahora los diez restaurantes con mayor similaridad al "Restaurante Pueblo Bonito" y con respecto a la métrica de similaridad de Pearson.**

### **Se deberá usar la cantidad de componentes necesarios para que la varabilidad de la factorización SVD quede explicada en un 90%.**

### **a.  Para ello se define primero la matriz de utilidad, UtMx_service, donde los renglones son los nombres de los restaurantes, las columnas los usuarios y las entradas las evaluaciones con respecto al servicio recibido en el restaurante.**





In [ ]:
# Se define la matriz de utilidad cuyos renglones sean los nombres de los
# restaurantes, las columnas los IDs de los usuarios y las entradas la
# evaluación del servicio (service_rating).


# Matriz de utilidad.
UtMx_service = df_combinado.pivot_table(values='service_rating', index='name', columns='userID', fill_value=0)

# Mostrar resultados.
print('Dimensión de la matriz de Utilidad sobre la evaluación del servicio:')
print('(restaurantes, usuarios) =', (UtMx_service.shape))
UtMx_service.head()

Dimensión de la matriz de Utilidad sobre la evaluación del servicio:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Arrachela Grill,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cabana Huasteca,0,0,2,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
Cafe Chaires,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cafeteria cenidet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Se incluye el código necesario para obtener y desplegar los 10 restaurantes con
# mayor similaridad de Pearson al "Restaurante Pueblo Bonito", y usando solo el 90%
# de los componentes que explican la varianza de la matriz UtMx_service.
# Se despliegan en orden descendente, sin incliur el restaurante de referencia.


# Número de componentes.
nc_service = UtMx_service.shape[1] - 1


# Utilizar TruncatedSVD para encontrar la cantidad de componentes que explican el 90% de la variabilidad.
SVD_service = TruncatedSVD(n_components=nc_service)
SVD_service.fit(UtMx_service)


# Encontrar el número de componentes necesario para explicar el 90% de la variabilidad.
for j in range(nc_service):
    if SVD_service.explained_variance_ratio_[0:j].sum() > 0.90:
        break
N_service = j - 1

# Mostrar resultados.
print('Total de valores singulares basados en evaluación del servicio:', nc_service)
print('Valor de truncamiento al 90% de dicha variabilidad:', N_service)
print()


# Volver a aplicar TruncatedSVD con el número óptimo de componentes.
SVD_service = TruncatedSVD(n_components=N_service)
resultant_matrix_service = SVD_service.fit_transform(UtMx_service)


# Calcular la matriz de correlación.
corr_mat_service = np.corrcoef(resultant_matrix_service)


# Encontrar las similitudes con el restaurante de referencia.
restaurante_de_referencia = "Restaurante Pueblo Bonito"
nombres_rest = UtMx_service.T.columns
idx_rest = list(nombres_rest).index(restaurante_de_referencia)
corr_rest = corr_mat_service[idx_rest]


# Buscar las correlaciones positivas.
idx = (corr_rest > 0)
mejores_sim_service = list()
for i in range(len(nombres_rest[idx])):
    mejores_sim_service.append((corr_rest[idx][i], nombres_rest[idx][i]))

print('Total de similaridades positivas encontradas:', len(mejores_sim_service))


# Lista de recomendaciones ordenada.
#mejores_sim_service_ordenadas = sorted(mejores_sim_service, key=lambda x:x[0], reverse=True)
mejores_sim_service_ordenadas = sorted(mejores_sim_service, reverse=True)


# Mostrar resultados.
print('Similitudes con base a la evaluación del servicio con mayores valores de correlación:')
for k in range(1, 11):  # Imprime los primeros 10 resultados ordenados.
    print('%d> %s' % (k, mejores_sim_service_ordenadas[k]))

Total de valores singulares basados en evaluación del servicio: 137
Valor de truncamiento al 90% de dicha variabilidad: 51

Total de similaridades positivas encontradas: 104
Similitudes con base a la evaluación del servicio con mayores valores de correlación:
1> (0.6644505135881374, 'pizza clasica')
2> (0.5858397267096588, 'El Club')
3> (0.4946814639241991, 'Hamburguesas saul')
4> (0.4696438137299943, 'Restaurante la Estrella de Dima')
5> (0.42995971873317684, 'Restaurante Guerra')
6> (0.4002031524450227, 'Restaurant Orizatlan')
7> (0.3844903092445727, 'Restaurante El Cielo Potosino')
8> (0.3542041714420721, 'la Cochinita Pibil Restaurante Yucateco')
9> (0.33769360159380507, 'La Fontana Pizza Restaurante and Cafe')
10> (0.3259320367095231, 'Restaurant Oriental Express')


## **Ejercicio - 7**

### **Incluye tus comentarios y conclusiones de la actividad. En particular indica cuáles restaurantes encontraste con correlaciones altas tanto para la calificación de la comida, como del servicio.**

Se empleó la reducción de dimensionalidad y la descomposición en valores singulares (SVD) para desarrollar un sistema de recomendación de restaurantes. Este sistema se utilizó para evaluar la similitud del "Restaurante Pueblo Bonito" con otros establecimientos con el propósito de hacer recomendaciones, empleando la métrica de similaridad de Pearson.

El sistema de recomendación se aplicó primero con un enfoque a la calificación de la comida y después se aplicó considerando la calificación del servicio.

Se identificaron varios restaurantes con una correlación mayor a 0.5 con el "Restaurante Pueblo Bonito" en cuanto a la calidad de la comida:

1. (0.7907229965411293, 'Restaurante la Estrella de Dima')
2. (0.7010299463961229, 'pizza clasica')
3. (0.6658698388504005, 'Restaurante Guerra')
4. (0.5451091923167452, 'El Club')


Se identificaron dos restaurantes con una correlación mayor a 0.5 con el "Restaurante Pueblo Bonito" en cuanto a la calidad del servicio:

1. (0.6650343354538824, 'pizza clasica')
2. (0.5874438554637492, 'El Club')


Teniendo en cuenta ambas calificaciones, "Pizza Clásica" y "El Club" destacaron al mostrar una alta correlación con el "Restaurante Pueblo Bonito", tanto en la calidad de la comida como en la del servicio. Estos dos establecimientos se posicionaron entre los cuatro primeros en el ranking de recomendaciones en ambas categorías.

Los resultados obtenidos en correlación para el restaurante "Pizza Clasica" fueron 0.7010299463961229 con respecto a la comida y 0.6650343354538824 con respecto al servicio. Los resultados obtenidos en correlación para el restaurante "El Club" fueron 0.5451091923167452 con respecto a la comida y 0.5874438554637492 con respecto al servicio. En base a los resultados anteriores se recomiendan los restaruantes "Pizza Clasica" y "El Club".